# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
db = pd.read_csv('Admission_Predict.csv')
db.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
5,6,321,109,3,3.0,4.0,8.20,1,0.75
6,7,308,101,2,3.0,4.0,7.90,0,0.68
7,8,302,102,1,2.0,1.5,8.00,0,0.50
8,9,323,108,3,3.5,3.0,8.60,0,0.45
9,10,327,111,4,4.0,4.5,9.00,1,0.84


Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
db.columns = db.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
db.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
db.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
db.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here
print(db.set_index(['GRE Score', 'CGPA']).index.is_unique)

True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
# your code here
db.set_index(['GRE Score','CGPA'], inplace=True)

In [9]:
db.reset_index(inplace = True)
db.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [10]:
# your code here
db2 = db[(db['CGPA'] > 9) & db['Research'] == 1]
db2.shape

(101, 9)

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# your code here
db3 = db[(db['CGPA'] > 9) & db['SOP'] < 3.5]
db3.shape

(385, 9)

In [12]:
db['Chance of Admit'].mean()

0.7239220779220773

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [13]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    the_mean = np.mean(col)
    the_std = np.std(col)
    return (col - the_mean)/the_std
    
 
        
        
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [14]:
# your code here
db['CGPA_std'] = standardize(db['CGPA'])
db['GRE_std'] = standardize(db['GRE Score'])
db['LOR_std'] = standardize(db['LOR'])
db.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [15]:
# Libraries
from random import choices

In [16]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=db.shape[0])
db.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [17]:
# your code here
db['deciding_column'] = db.lookup(db.index, decision_choice)
db.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [18]:
# your code here
db['decision'] = np.where(db['deciding_column'] > 0.8, '1', '0')
db.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [19]:
# your code here
db['decision'].value_counts()

0    302
1     83
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [20]:
# your code here
db.columns = db.columns.str.replace('[^\w\s]','').str.replace('\s','_').str.lower()
db.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [21]:
db['adjusted_gre'] = np.where(db['university_rating'] >= 4, db['gre_score'] + 10, db['gre_score'])

In [23]:
db['adjusted_gre'] = pd.cut(db['adjusted_gre'],4)

In [24]:
db.head(20)

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1,"(335.0, 350.0]"
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0,"(305.0, 320.0]"
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,"(320.0, 335.0]"
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,"(305.0, 320.0]"
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0,"(335.0, 350.0]"
5,321,8.20,6,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,-0.660055,0,"(320.0, 335.0]"
6,308,7.90,7,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,-1.158723,0,"(305.0, 320.0]"
7,302,8.00,8,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,-1.276191,0,"(289.94, 305.0]"
8,323,8.60,9,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,0.542921,0,"(320.0, 335.0]"
9,327,9.00,10,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,1.193197,1,"(335.0, 350.0]"
